In [ ]:
 %%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

### Get Images

In [ ]:
import gdown
import zipfile

# https://drive.google.com/file/d/1YgDh3Vy3bqmZYP5XK2gpkCHsITm2xLRg/view?usp=sharing


file_id = "1YgDh3Vy3bqmZYP5XK2gpkCHsITm2xLRg"
output = "train_images_170625.zip"

# Download the file
gdown.download(f"https://drive.google.com/uc?id={file_id}", output, quiet=False)

with zipfile.ZipFile(output, 'r') as zip_ref:
    zip_ref.extractall("train_images")

# Remove the zip file
os.remove(output)

Downloading...
From (original): https://drive.google.com/uc?id=1YgDh3Vy3bqmZYP5XK2gpkCHsITm2xLRg
From (redirected): https://drive.google.com/uc?id=1YgDh3Vy3bqmZYP5XK2gpkCHsITm2xLRg&confirm=t&uuid=1ab494cb-ed3f-4bb3-8dc4-68bf82cec781
To: /content/train_images_170625.zip
100%|██████████| 815M/815M [00:16<00:00, 50.4MB/s]


### Get CoT Outputs

In [ ]:
import requests
import pandas as pd

# Get deepseek r1 generated CoT captions of CT and MR images.
url = "https://raw.githubusercontent.com/canatess/RAD-ACE/refs/heads/main/cot_generated_deepseek.csv"

response = requests.get(url)

# Save responses as files
with open("df.csv", "wb") as f:
    f.write(response.content)


df = pd.read_csv("df.csv")

df.head()

,filename,modality,cot_output,question
0,['images/pmc_1503_0.jpg'],Computed Tomography,"<think>\nAlright, let's break down what I'm se...",Describe the structures and notable features i...
1,['images/pmc_1504_0.jpg'],Computed Tomography,"<think>\nOkay, so I'm looking at a chest CT sc...",Analyze the image in a comprehensive and detai...
2,['images/pmc_1505_0.jpg'],Computed Tomography,"<think>\nOkay, so I'm trying to figure out wha...",Analyze the image in a comprehensive and detai...
3,['images/pmc_1506_0.jpg'],Magnetic Resonance Imaging,"<think>\nOkay, I need to explain the visual co...",What details stand out in this image?
4,['images/pmc_1507_0.jpg'],Computed Tomography,"<think>\nAlright, I need to analyze the given ...",Provide a step-by-step interpretation of the i...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16800 entries, 0 to 16799
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   filename    16800 non-null  object
 1   modality    16800 non-null  object
 2   cot_output  16800 non-null  object
 3   question    16800 non-null  object
dtypes: object(4)
memory usage: 525.1+ KB


In [ ]:
# Convert DataFrame to list of dictionaries
dataset = df.to_dict(orient="records")

In [ ]:
import ast
import os
from PIL import Image
from concurrent.futures import ThreadPoolExecutor

def process_image(path):
    try:
        full_path = os.path.join("train_images/train_images", os.path.basename(path))
        image = Image.open(full_path).convert("RGB")
        image.thumbnail((296, 296))  # Faster resizing
        return image
    except Exception:
        return None

def convert_to_conversation(sample):
    question = sample["question"]
    cot_output = sample["cot_output"]

    try:
        image_paths = ast.literal_eval(sample["filename"])
    except (SyntaxError, ValueError):
        return None

    with ThreadPoolExecutor() as executor:
        images = list(filter(None, executor.map(process_image, image_paths)))

    if not images:
        return None

    conversation = [
        {"role": "user", "content": [{"type": "text", "text": question}, {"type": "image", "image": images[0]}]},
        {"role": "assistant", "content": [{"type": "text", "text": cot_output}]},
    ]
    return {"messages": conversation}

In [ ]:
# Example formatted data for Qwen model
conversation = convert_to_conversation(dataset[0])
conversation

{'messages': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Describe the structures and notable features in the image.'},
    {'type': 'image',
     'image': <PIL.Image.Image image mode=RGB size=296x274>}]},
  {'role': 'assistant',
   'content': [{'type': 'text',
     'text': '<think>\nAlright, let\'s break down what I\'m seeing here. The image provided is a CT scan focusing on the cervical spine, specifically looking at three different planes: axial, sagittal, and coronal. That gives me a comprehensive view of the area from various angles, which is crucial for accurately diagnosing any abnormalities.\n\nLooking at the axial slices first, I notice there\'s a fracture at the base of the odontoid process. The term "mildly comminuted" suggests that the fracture isn\'t too fragmented; it\'s just a small break. Now, moving on to the sagittal view, this gives me a better sense of alignment and any potential displacement. Here, I see that there\'s less than 2 mm of anterolist

In [ ]:
# Process all the dataset takes around 1-2 min
from tqdm import tqdm

converted_dataset = []
for sample in tqdm(dataset, desc="Converting all"):
    converted = convert_to_conversation(sample)
    if converted is not None:
        converted_dataset.append(converted)

Converting all: 100%|██████████| 16800/16800 [01:16<00:00, 219.45it/s]


In [ ]:
print("Length of conversation formatted dataset : ",len(converted_dataset))

Length of conversation formatted dataset :  12233


### Download Base Llama3.2-11B Model

In [ ]:
from unsloth import FastVisionModel
model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/Llama-3.2-11B-Vision-Instruct",
    load_in_4bit = True, # Use 4bit to reduce memory use. False for 16bit LoRA.
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.2: Fast Mllama patching. Transformers: 4.52.4.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Setting Up LoRa For Efficient Fine-Tuning

In [ ]:
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = True, # False if not finetuning vision layers
    finetune_language_layers   = True, # False if not finetuning language layers
    finetune_attention_modules = True, # False if not finetuning attention layers
    finetune_mlp_modules       = True, # False if not finetuning MLP layers

    r = 16,           # The larger, the higher the accuracy, but might overfit
    lora_alpha = 16,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
    # target_modules = "all-linear", # Optional now! Can specify a list if needed
)

Unsloth: Making `model.base_model.model.model.vision_model.transformer` require gradients


### Training Parameters and HyperParameters

In [ ]:
from unsloth import is_bf16_supported, FastVisionModel
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig

# Enable training optimizations
FastVisionModel.for_training(model)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=UnslothVisionDataCollator(model, tokenizer),  # Handles image + text inputs
    train_dataset=converted_dataset,
    args=SFTConfig(
        per_device_train_batch_size=2,        # Batch size per GPU
        gradient_accumulation_steps=4,        # Accumulate grads to simulate larger batch size
        warmup_steps=50,                      # LR warmup steps at start of training
        max_steps =2200,                   # Number of passes over dataset

        learning_rate=2e-4,                   # Initial learning rate
        fp16=not is_bf16_supported(),         # Use fp16 if bf16 is not supported
        bf16=is_bf16_supported(),             # Prefer bf16 if hardware supports it
        logging_steps=10,                     # Log every 10 steps
        optim="adamw_8bit",                   # 8-bit optimizer for memory efficiency

        weight_decay=0.01,                    # Regularization to prevent overfitting
        lr_scheduler_type="cosine",           # Cosine learning rate decay

        seed=3407,                            # For reproducibility
        output_dir="outputs",                 # Where to save checkpoints and logs
        report_to="none",                     # Disable W&B or HF tracking

        remove_unused_columns=False,          # Required for multimodal (image+text) training
        dataset_text_field="",                # Unused for vision; leave empty
        dataset_kwargs={"skip_prepare_dataset": True},  # Skip preprocessing (dataset already ready)
        dataset_num_proc=4,                   # Parallel processing workers
        max_seq_length=2048,                  # Max context length (e.g., Qwen2.5-VL)
    )
)


### Training

In [ ]:
# Begin training
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 12,233 | Num Epochs = 2 | Total steps = 2,200
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 67,174,400/11,000,000,000 (0.61% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.725100
20,1.545200
30,1.309700
40,1.232600
50,1.154400
60,1.137200
70,1.101700
80,1.100100
90,1.060900
100,1.057600


### Save Model

In [ ]:
# Save model weights in a folder
model.save_pretrained("llama11b")
tokenizer.save_pretrained("llama11b")

[]

In [ ]:
# Download zipped model into local computer
!zip -r llama11b.zip llama11b

from google.colab import files
files.download("llama11b.zip")

  adding: llama11b/ (stored 0%)
  adding: llama11b/tokenizer_config.json (deflated 96%)
  adding: llama11b/special_tokens_map.json (deflated 71%)
  adding: llama11b/adapter_config.json (deflated 56%)
  adding: llama11b/tokenizer.json (deflated 85%)
  adding: llama11b/chat_template.jinja (deflated 72%)
  adding: llama11b/preprocessor_config.json (deflated 49%)
  adding: llama11b/adapter_model.safetensors (deflated 7%)
  adding: llama11b/README.md (deflated 66%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Get Test Images

In [ ]:
gdown.download("https://drive.google.com/uc?id=18nb9I1ff6YChu7X4H_OPeax4DmefePdJ", "images.zip", quiet=False)

with zipfile.ZipFile("images.zip", "r") as zip_ref:
    zip_ref.extractall("test_images")

image_dir = "test_images/photos"

Downloading...
From: https://drive.google.com/uc?id=18nb9I1ff6YChu7X4H_OPeax4DmefePdJ
To: /content/images.zip
100%|██████████| 1.00M/1.00M [00:00<00:00, 31.1MB/s]


### Generate Reports On Test Data and Save CSV

In [ ]:
# Run through all and download the reports generated

def run_inference_on_image(model, tokenizer, image_path, instruction="Describe this image as if you are an radiologist"):
    image = Image.open(image_path).convert("RGB")
    image.thumbnail((296, 296))

    messages = [
        {"role": "user", "content": [
            {"type": "image"},
            {"type": "text", "text": instruction}
        ]}
    ]

    input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
    inputs = tokenizer(image, input_text, add_special_tokens=False, return_tensors="pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens=2048, use_cache=True)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Remove system instruction
    decoded = decoded.split("assistant")[-1].strip()

    return decoded

In [ ]:
results = []

# Get image filenames
image_files = [f for f in os.listdir(image_dir) if f.lower().endswith((".jpg", ".jpeg", ".png"))]

for fname in tqdm(image_files, desc="Running inference on images"):
    img_path = os.path.join(image_dir, fname)
    output = run_inference_on_image(model, tokenizer, img_path)
    results.append({"image_name": fname, "model_output": output})

# Save to CSV
df = pd.DataFrame(results)
df.to_csv("llama11b_inference_results.csv", index=False)

# Download CSV
from google.colab import files
files.download("llama11b_inference_results.csv")

Running inference on images: 100%|██████████| 25/25 [48:38<00:00, 116.74s/it]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>